In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import csv

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
df

In [ ]:
train_df = df.loc[:20896]
val_df = df.loc[20897:].reset_index(drop = True)

In [ ]:
train_df['image_id'] = '../input/cassava-leaf-disease-classification/train_images'+os.sep + train_df['image_id']
val_df['image_id'] = '../input/cassava-leaf-disease-classification/train_images'+os.sep + val_df['image_id']

In [ ]:
train_files_ds = tf.data.Dataset.from_tensor_slices((train_df['image_id'],train_df['label']))
val_files_ds = tf.data.Dataset.from_tensor_slices((val_df['image_id'],val_df['label']))

In [ ]:
CLASSES = 5
EPOCHS = 30
BATCH_SIZE = 10
AUG_BATCH = BATCH_SIZE
IMAGE_SIZE = (224,224)
NUM_TRAINING_IMAGES = len(train_df)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
def handle_filenames(filename,label):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image,channels = 3)
    image = tf.cast(image,tf.float32)/255.0
    image = tf.image.resize(image,size = (224,224))
    return image,tf.one_hot(label,CLASSES)


train_ds = train_files_ds.map(handle_filenames).repeat().batch(BATCH_SIZE).prefetch(AUTO)
val_ds = val_files_ds.map(handle_filenames).batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1)
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


In [ ]:
def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2

In [ ]:
row = 6; col = 4;
row = min(row,AUG_BATCH//col)
all_elements = train_ds.unbatch()
augmented_element = all_elements.repeat().batch(AUG_BATCH).map(cutmix)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:
row = 6; col = 4;
row = min(row,AUG_BATCH//col)
all_elements = train_ds.unbatch()
augmented_element = all_elements.repeat().batch(AUG_BATCH).map(mixup)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:
def transform(image,label):
    DIM = IMAGE_SIZE[0]
    CLASSES = 5
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4

In [ ]:
row = 6; col = 4;
row = min(row,AUG_BATCH//col)
all_elements = train_ds.unbatch()
augmented_element = all_elements.repeat().batch(AUG_BATCH).map(transform)

for (img,label) in augmented_element:
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:
preprocess = keras.Sequential([
    layers.RandomFlip(),
    layers.RandomRotation(0.2),
    layers.RandomZoom((-0.2, 0)),
    layers.RandomContrast((0.2,0.2))
])

#vit_backbone = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_b8_fe/1", trainable=False) 
backbone = keras.applications.EfficientNetB3(
    include_top = False,
    weights = '../input/efficientnetb3-notop/efficientnetb3_notop.h5',
    input_shape = (224,224,3)
)
resnet_preprocess = keras.applications.efficientnet.preprocess_input
backbone.trainable = True

In [ ]:
decay_steps = int(round(len(train_df)/10))*10
cosine_decay = keras.experimental.CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)

In [ ]:
with strategy.scope():
    inp = keras.Input(shape = (224,224,3))
    x = preprocess(inp)
    x = resnet_preprocess(x)
    x = backbone(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(5,activation = 'softmax',dtype = tf.float32)(x)
    model = keras.Model(inp,x)
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = keras.optimizers.Adam(cosine_decay),
        metrics = ['categorical_accuracy']
    )


In [ ]:
LR_START = 0.0001
LR_MAX = 0.001 * strategy.num_replicas_in_sync
LR_MIN = 0.0001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
aug_train = train_ds.unbatch()
aug_train = aug_train.repeat().batch(AUG_BATCH).map(transform)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,
    restore_best_weights = True
)
model.fit(aug_train,
          validation_data = val_ds,
          epochs = 30,
          steps_per_epoch = STEPS_PER_EPOCH,
          callbacks = [early_stopping])

In [ ]:
model.save('aug_model.h5')

In [ ]:
l = os.listdir("../input/cassava-leaf-disease-classification/test_images")
parent = "../input/cassava-leaf-disease-classification/test_images/"
predictions = []
predictions.append(["image_id", "label"])
for i in l :
    child = parent + i
    img = cv2.imread(child)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    img = img.reshape((1, 224, 224, 3))
    pred = model.predict(img)
    pred = pred.reshape((5,))
    print(np.argmax(pred))
    del img
    predictions.append([i, str(np.argmax(pred))])
    del pred


with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predictions)

In [ ]:
pd.read_csv('submission.csv')